In [ ]:
from collections import defaultdict
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
from os.path import join, basename, isdir
from os import listdir, makedirs
import shutil as sh
import pickle

# Sort Trials

In [ ]:
HOME = os.environ["HOME"]
os.chdir(join(HOME, 'tmp/<run-name>/'))

In [ ]:
dirs = list(filter(lambda d: isdir(d), listdir('.')))
dirs = list(filter(lambda d: 'neural' in d, dirs))
durations = {}
durations['private'] = {'nc': defaultdict(list), 'nn': defaultdict(list)}
durations['shared'] = {'nc': defaultdict(list), 'nn': defaultdict(list)}
for d in dirs:
    t = float(open(join(d, 'duration'), 'r').readline().strip())
    preamble = "shared" if "shared" in d else "private"
    agents = "nc" if "classic" in d else "nn"
    durations[preamble][agents][t].append(d)
for k in durations["shared"]["nc"]:
    durations["shared"]["nc"][k].sort()
for k in durations["shared"]["nn"]:
    durations["shared"]["nn"][k].sort()
for k in durations["private"]["nc"]:
    durations["private"]["nc"][k].sort()
for k in durations["private"]["nn"]:
    durations["private"]["nn"][k].sort()
print(durations["private"]["nn"])

# Trial Outcomes

In [ ]:
def success_rate(ms): 
    if ms.size <= 40:
        return sum(ms < 0.1) / 40.
    else:
        return sum(ms < 0.1) / 66.

def do_runtime(dirs, sharedstr, agentstr, timestr):
    mins = []
    means = []
    maxes = []
    for d in dirs:
        try:
            with open(join(d, 'results'), 'r') as f:
                lcount, lmean, lmin, lmax, _ = f.readlines()
                mins.append(float(lmin.strip().split()[-1]))
                means.append(float(lmean.strip().split()[-1]))
                maxes.append(float(lmax.strip().split()[-1]))
        except:
            pass

    plt.stem(means)
    plt.title("{} preamble {}, {} seconds trials".format(
                sharedstr, agentstr, timestr))
    plt.show()
    print("Success rate", success_rate(np.array(means)))
    return mins, means, maxes

for sharedstr in ("shared", "private"):
    for agentstr in ("nn", "nc"):
        print(' '.join(["=====", sharedstr.upper(), 
                       agentstr.upper(), "====="]))
        for time in durations[sharedstr][agentstr]:
            do_runtime(durations[sharedstr][agentstr][time],
                       sharedstr, agentstr, time)

# Constellations

In [ ]:
for d in durations['private']['nn'][500]:
    final_mod = sorted(list(filter(lambda d: 'neural_mod' in d and 'npy' in d, listdir(d))))[-1]
    const = np.load(join(d, final_mod))
    plt.scatter(const.real, const.imag, label="Constellation")
    plt.scatter([np.mean(const).real], [np.mean(const).imag], label="Constellation Center", marker='*')
    plt.gca().add_artist(plt.Circle((0,0), 1./np.sqrt(2), fill=False, linestyle='--', label="Unit Circle"))
#     plt.gca().add_artist(plt.Rectangle((-1/np.sqrt(2),-1/np.sqrt(2)), 2/np.sqrt(2), 2/np.sqrt(2), fill=False, color='g', linewidth=1.5, label="Tx Bound"))
    plt.gca().add_artist(plt.Rectangle((-1,-1), 2, 2, fill=False, color='r', linewidth=1.5, label="Tx Bound"))
    plt.title(d)
    plt.legend()
    plt.xlim([-1.5, 1.5])
    plt.ylim([-1.5, 1.5])
    plt.gca().set_aspect('equal')
    plt.grid()
    plt.show()

# Trained Symbols vs BER

In [ ]:
def do_symbols_ber(durations, sharedstr, agentstr):
    mins = []
    means = []
    maxes = []
    symbols = []
    for t in durations:
        for d in durations[t]:
            try:
                with open(join(d, 'results'), 'r') as f:
                    lcount, lmean, lmin, lmax, lsymbs = f.readlines()
                    mins.append(float(lmin.strip().split()[-1]))
                    means.append(float(lmean.strip().split()[-1]))
                    maxes.append(float(lmax.strip().split()[-1]))
                    symbols.append(int(lsymbs.strip().split()[-1]))
            except:
                pass
    mins = np.array(mins)
    means = np.array(means)
    maxes = np.array(maxes)
    symbols = np.array(symbols)
    if len(means) > 0:
        plt.scatter(symbols / 1e6, means)
        plt.title("{} preamble {} trials".format(
                    sharedstr, agentstr))
        plt.xlabel("MSymbols")
        plt.ylabel("BER")
        plt.show()
    return mins, means, maxes, symbols

for sharedstr in ("shared", "private"):
    for agentstr in ("nn", "nc"):
        do_symbols_ber(durations[sharedstr][agentstr], 
                       sharedstr, agentstr)

In [ ]:
def success_rate(ms): 
    if ms.size <= 20:
        return sum(ms < 0.07) / 20.
    if ms.size <= 40:
        return sum(ms < 0.07) / 40.
    else:
        return sum(ms < 0.07) / 66.
    

# OPTIONAL: get equivalent classic snr
snr0 = 10.56335866
with open("../snr-runs/gain-snr-ber-coeffs.pkl", "rb") as f:
    coeffs = pickle.load(f)
gain2snr = lambda x: coeffs['gain-snr-linear'][0] * x + coeffs['gain-snr-linear'][0]
snr2ber = lambda x: np.exp(coeffs['snr-ber-logquadratic'][0] * x ** 2 + 
                           coeffs['snr-ber-logquadratic'][1] * x + 
                           coeffs['snr-ber-logquadratic'][2])
def ber2snr(ber):
    guess = 10
    past = 0
    while guess > 0 and guess < 20:
        if snr2ber(guess) <= ber:
            guess -= 0.01
            if past == +1:
                break
            past = -1
        else:
            guess += 0.01
            if past == -1:
                break
            past = +1
    return guess

def success_rate_snr(snrs, threshold):
    if snrs.size <= 20:
        return sum(snr0 - snrs < threshold) / 20.
    if snrs.size <= 40:
        return sum(snr0 - snrs < threshold) / 40.
    else:
        return sum(snr0 - snrs < threshold) / 66.
    
print("3dB BER:", snr2ber(snr0-3))
print("5dB BER:", snr2ber(snr0-5))
# OPTIONAL 

    
def do_success_rate(durations, sharedstr, agentstr, threshold=3):
    mean_symbs = []
    srate = []
    for t in durations:
        mins = []
        means = []
        maxes = []
        symbols = []
        for d in durations[t]:
            try:
                with open(join(d, 'results'), 'r') as f:
                    lcount, lmean, lmin, lmax, lsymbs = f.readlines()
                    mins.append(float(lmin.strip().split()[-1]))
                    means.append(float(lmean.strip().split()[-1]))
                    maxes.append(float(lmax.strip().split()[-1]))
                    symbols.append(int(lsymbs.strip().split()[-1]))
            except:
                pass
        mean_symbs.append(np.mean(symbols))
        equiv_snrs = np.array([ber2snr(m) for m in means])
        srate.append(success_rate_snr(equiv_snrs, threshold))
    mean_symbs = np.array(mean_symbs)
    srate = np.array(srate)
    isort = np.argsort(mean_symbs)
    mean_symbs = mean_symbs[isort]
    srate = srate[isort]
    if len(srate) > 0:
        plt.stem(mean_symbs / 1e6, srate * 100)
        plt.title("{} preamble {} trials".format(
                    sharedstr, agentstr))
        plt.xlabel("MSymbols")
        plt.ylabel("Trials Within 3dB BER (%)")
        plt.xlim([0,2])
        plt.ylim([0,100])
        plt.show()
    return mean_symbs, srate


for sharedstr in ("shared", "private"):
    for agentstr in ("nn", "nc"):
        symbs, success3 = do_success_rate(durations[sharedstr][agentstr], 
                                         sharedstr, agentstr, 3)
        symbs, success5 = do_success_rate(durations[sharedstr][agentstr], 
                                         sharedstr, agentstr, 5)
        out = {'symbols_exchanged': list(symbs),
               'success_rate_3dB': list(success3),
               'success_rate_5dB': list(success5)}
        with open("{}-{}-seed_convergence.pkl".format(sharedstr, agentstr), "wb") as f:
            pickle.dump(out, f)

# Retest Setup

In [ ]:
# Copy final mod & demod models
makedirs('./trained-models1', exist_ok=True)
makedirs('./trained-models2', exist_ok=True)
with open("trained-models1/dirlist.txt", 'w') as f:
    for shared in "shared", "private":
        for agents in "nn", "nc":
            for t in durations[shared][agents]:
                print(shared, agents, t)
                for d in durations[shared][agents][t]:
                    dirname = d[:-5]
                    makedirs(join('./trained-models1', dirname), exist_ok=True)
                    makedirs(join('./trained-models2', dirname), exist_ok=True)
                    with open(join("trained-models1", dirname, "shared"), "w") as fsh:
                        fsh.write(shared)
                    with open(join("trained-models1", dirname, "mode"), "w") as fmo:
                        fmo.write(agents) 
                    with open(join("trained-models1", dirname, "duration"), "w") as fdu:
                        fdu.write(str(t)) 
                    with open(join("trained-models2", dirname, "shared"), "w") as fsh:
                        fsh.write(shared)
                    with open(join("trained-models2", dirname, "mode"), "w") as fmo:
                        fmo.write(agents) 
                    with open(join("trained-models2", dirname, "duration"), "w") as fdu:
                        fdu.write(str(t)) 
                    if 'srn1' in d:
                        try:
                            sh.copy(glob.glob(join(d, 'mod_*.mdl'))[0], join('trained-models1', dirname, 'mod-weights.mdl'))
                            sh.copy(glob.glob(join(d, 'demod_*.mdl'))[0], join('trained-models1', dirname, 'demod-weights.mdl'))
                        except IndexError as e:
                            print("***", dirname)
                            continue
                        f.write(dirname + "\n")
                    if 'srn2' in d:
                        try:
                            sh.copy(glob.glob(join(d, 'mod_*.mdl'))[0], join('trained-models2', dirname, 'mod-weights.mdl'))
                            sh.copy(glob.glob(join(d, 'demod_*.mdl'))[0], join('trained-models2', dirname, 'demod-weights.mdl'))
                        except IndexError:
                            pass

sh.copy("./trained-models1/dirlist.txt", "./trained-models2/dirlist.txt")